# <center>Project: Beware Bunnies</center>

### Michael Salamon, Dor Ulman, Nahin Imtiaz, Asma Sadia

##### CSC 59866 Senior Design Project I - Applied Machine Learning
##### The City College of New York

---

###### opening paragraphs will go here

-          What is the problem that you’re solving? Who are you solving it for?  

	The European Rabbit (Oryctolagus cuniculus) was translocated to Australia in the 1880’s as a food resource and for hunting activities. This species of rabbit spread rapidly across Australia and became a major issue for the locals. Because of its reproductive capacity, the rabbit community increased rapidly, invading and exploiting a diversity of green areas and disrupted the ecosystems in certain regions. Currently, the European rabbit is present in more than 70% of the Australian continent, surpassing the growth of the local vegetation and competing with local animals for food and habitants. The rabbits are also a source of food for predators which inflate their population and for these reasons the economy has 200 million loss for the Australian economy annually. Predicting the growth rate and regions of the invasive species will allow the Australian government to allocate means efficiently in order to regulate the European Rabbit population. Means of regulation include lacing rabbit food w/ viruses or sending specially trained dogs to certain areas. The model should be accurate enough for helping the Australian continent to send resources to areas with far too many rabbits to regulate the population to give time for vegetation/ecosystem to grow back.
    
-          What are potential features, whether already built or that you’ll need to make? What is your target variable?  

	Potential features(from abundance dataset): 
    RabbitsPerKm - number of rabbits observed per kilometre
    Season or Date(Day, Month, Year) - measurement of time.
    DistPermWater - Euclidean weighted distance to permanent water features(km)
    DistAgrilLand - Euclidean distance to agricultural land margins(km)
    Location - Longitude and latitude (maybe also state?)
	Target variables:
    Rabbit population prediction (per km)
    Location in which the growth happen

-          What transformations will you need to perform on your input data?  
While looking at the datasets statistics (using .describe() on the data frame), some rows are missing data and will probably be removed. For example, in both datasets, columns ‘Month’ and ‘Day’ are missing values which are crucial for finding accurate trends of rabbit’s population growth over time. In addition, taking into consideration the accuracy of collected data, meaning filtering data that was not accurately collected - “QA_Accuracy” which is the ‘quality assessment of the data accuracy’.

-          What modeling approach(es) will you use for your problem? Is this classification or regression? 
Regression  -  Two ML models that are ideal to predict population growth are k-Nearest Neighbors (kNN) and Random Forest.
Math approach - https://mathinsight.org/controlling_rabbit_population 
-          How will you evaluate your model’s performance? This should be in terms of quantitative metrics, qualitative evaluation, and visualizations.
Comparison with math models.


In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 99% of the data in occurence takes place in the past 35 years. (Roy-Dufresne 6)
occurence = pd.read_csv('SI1_DataOccurrence.csv')
occurence

,Occurrence_ID,AustraliaUTMGrid10Km_ID,AustraliaUTMGrid10Km_Lat,AustraliaUTMGrid10Km_Long,ResolutionQuality,TotalNbOccurrencesPerUTMGrid,PtsIntensityEstimate_AllDataPerUTMGrid,PtsIntensityEstimate_OneDataPerUTMGrid,AustraliaState,DataType,...,A_Prec_AvgAutumn30Yr,A_Prec_AvgSummer30Yr,A_Prec_AvgSpring30Yr,A_Prec_AvgWinter30Yr,VegeType,DistPermWater,DistAgriLand,PercSoilClay,MinDayLength,VarDayLength
0,1,55H1782,-34.95,148.15,Less_than_5km,8,0.23,0.90,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,11.0,1.48,7.32,16.33,9.8,2.63
1,2,55H1782,-34.95,148.15,Less_than_5km,8,0.23,0.90,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,11.0,1.48,7.32,16.33,9.8,2.63
2,3,55H1782,-34.95,148.15,Less_than_5km,8,0.23,0.90,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,11.0,1.48,7.32,16.33,9.8,2.63
3,4,55H1782,-34.95,148.15,Less_than_5km,8,0.23,0.90,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,11.0,1.48,7.32,16.33,9.8,2.63
4,5,55H1782,-34.95,148.15,Less_than_5km,8,0.23,0.90,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,11.0,1.48,7.32,16.33,9.8,2.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689260,689281,56K4693,-23.85,151.25,More_than_15km,1,0.01,0.21,SA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689261,689282,50H1833,-35.00,117.29,More_than_15km,3,0.00,0.14,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689262,689283,56H0842,-33.45,150.15,More_than_15km,25,0.11,0.74,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689263,689284,55H0350,-32.55,148.95,More_than_15km,3,0.16,0.80,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
abundance = pd.read_csv('SI2_DataAbundance.csv', low_memory=False)
abundance

,SpotData_ID,AustraliaUTMGrid10Km_ID,AustraliaUTMGrid10Km_Long,AustraliaUTMGrid10Km_Lat,ResolutionQuality,TotalNbTransectsPerUTMGrid,AustraliaState,StudyRegions_ID,DataSourceEntry_ID,DataSourceName,...,A_Prec_AvgAutumn30Yr,A_Prec_AvgSummer30Yr,A_Prec_AvgSpring30Yr,A_Prec_AvgWinter30Yr,VegeType,DistPermWater,DistAgriLand,PercSoilClay,MinDayLength,VarDayLength
0,1,55H0894,149.35,-33.45,Less_than_5km,3,NSW,3801,AUS_Published_Spot_001,AUS_Published_Spotlight,...,50.32,74.79,68.15,64.75,11,0.44,1.61,15.83,9.93,2.35
1,2,55H0412,149.15,-32.65,Less_than_5km,4,NSW,3805,AUS_Published_Spot_002,AUS_Published_Spotlight,...,49.00,67.26,61.46,49.31,12,1.19,2.23,18.05,10.00,2.20
2,3,56J3074,151.35,-29.15,Less_than_5km,2,QLD,3850,AUS_Published_Spot_003,AUS_Published_Spotlight,...,44.16,89.25,63.26,35.91,11,0.85,3.08,22.72,10.28,1.66
3,4,56J3074,151.35,-29.15,Less_than_5km,2,QLD,3850,AUS_Published_Spot_004,AUS_Published_Spotlight,...,44.16,89.25,63.26,35.91,11,0.85,3.08,22.72,10.28,1.66
4,5,56J1025,150.45,-25.75,Less_than_5km,2,QLD,3851,AUS_Published_Spot_005,AUS_Published_Spotlight,...,41.66,84.77,54.33,28.62,11,2.02,16.28,42.96,10.53,1.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51236,51237,54J4747,138.65,-31.95,Between_10km_and_15km,1,SA,3720,Yednalue_33,SA_YednalueVenus,...,22.39,27.30,31.59,31.74,4,4.52,14.11,28.91,10.06,2.09
51237,51238,54J4747,138.65,-31.95,Between_10km_and_15km,1,SA,3720,Yednalue_34,SA_YednalueVenus,...,22.39,27.30,31.59,31.74,4,4.52,14.11,28.91,10.06,2.09
51238,51239,54J4747,138.65,-31.95,Between_10km_and_15km,1,SA,3720,Yednalue_35,SA_YednalueVenus,...,22.39,27.30,31.59,31.74,4,4.52,14.11,28.91,10.06,2.09
51239,51240,54J4747,138.65,-31.95,Between_10km_and_15km,1,SA,3720,Yednalue_36,SA_YednalueVenus,...,22.39,27.30,31.59,31.74,4,4.52,14.11,28.91,10.06,2.09


In [5]:
#Listing the dataset columns
occurence.columns

Index(['Occurrence_ID', 'AustraliaUTMGrid10Km_ID', 'AustraliaUTMGrid10Km_Lat',
       'AustraliaUTMGrid10Km_Long', 'ResolutionQuality',
       'TotalNbOccurrencesPerUTMGrid',
       'PtsIntensityEstimate_AllDataPerUTMGrid',
       'PtsIntensityEstimate_OneDataPerUTMGrid', 'AustraliaState', 'DataType',
       'DataSourceEntry_ID', 'DataSourceName', 'DataSourceSiteSectionName',
       'Year', 'Month', 'Day', 'Season', 'QA_Accuracy', 'A_Prec_Avg30Yr',
       'A_Psea_Avg30Yr', 'A_TAvg_Avg30Yr', 'A_TMax_Avg30Yr', 'A_TMin_Avg30Yr',
       'A_TSea_Avg30Yr', 'A_TWet_Avg30Yr', 'A_TWrm_Avg30Yr',
       'A_Prec_AvgAutumn30Yr', 'A_Prec_AvgSummer30Yr', 'A_Prec_AvgSpring30Yr',
       'A_Prec_AvgWinter30Yr', 'VegeType', 'DistPermWater', 'DistAgriLand',
       'PercSoilClay', 'MinDayLength', 'VarDayLength'],
      dtype='object')

In [6]:
#Listing the datatypes of each column
occurence.dtypes

Occurrence_ID                               int64
AustraliaUTMGrid10Km_ID                    object
AustraliaUTMGrid10Km_Lat                  float64
AustraliaUTMGrid10Km_Long                 float64
ResolutionQuality                          object
TotalNbOccurrencesPerUTMGrid                int64
PtsIntensityEstimate_AllDataPerUTMGrid    float64
PtsIntensityEstimate_OneDataPerUTMGrid    float64
AustraliaState                             object
DataType                                   object
DataSourceEntry_ID                         object
DataSourceName                             object
DataSourceSiteSectionName                  object
Year                                      float64
Month                                     float64
Day                                       float64
Season                                    float64
QA_Accuracy                               float64
A_Prec_Avg30Yr                            float64
A_Psea_Avg30Yr                            float64


In [7]:
# There are many more weather columns for this dataset compared w/ the
# last dataset. I don't think these extra weather columns are unique
# just more specific versions of other weather columns... they can 
# probably be removed to get the same number of features as the
# occurence dataset.
abundance.columns

Index(['SpotData_ID', 'AustraliaUTMGrid10Km_ID', 'AustraliaUTMGrid10Km_Long',
       'AustraliaUTMGrid10Km_Lat', 'ResolutionQuality',
       'TotalNbTransectsPerUTMGrid', 'AustraliaState', 'StudyRegions_ID',
       'DataSourceEntry_ID', 'DataSourceName', 'DataSourceSiteSectionName',
       'VisitNb', 'TotalNbReplicates', 'Year', 'Month', 'Day', 'Season',
       'Transect_ID', 'TransectWidth', 'TransectLength', 'RabbitsPerKm',
       'UncertaintyManagementNotes', 'QA_Accuracy', 'QA_Completeness',
       'QA_TemporalCoverage', 'QA_Consistency', 'D_Prec', 'D_Prec_30DaysLag',
       'M_Prec', 'M_Prec_12PreMonths', 'M_Prec_24PreMonths',
       'M_Prec_2PresSeasons', 'A_Prec', 'A_Prec_1YrLag', 'A_Prec_2YrLag',
       'A_Psea', 'A_Psea_1YrLag', 'A_Psea_2YrLag', 'D_Tmax', 'D_Tmin',
       'M_TAvg_PreSummer', 'M_TAvg_PreWinter', 'A_AvgTMaxWarmestMonth',
       'A_AvgTMaxWarmestMonth_1YrLag', 'A_AvgTMinColdestMonth',
       'A_AvgTMinColdestMonth_1YrLag', 'A_Prec_Avg30Yr', 'A_Psea_Avg30Yr',
    

In [8]:
#Listing the datatypes of each column
abundance.dtypes

SpotData_ID                    int64
AustraliaUTMGrid10Km_ID       object
AustraliaUTMGrid10Km_Long    float64
AustraliaUTMGrid10Km_Lat     float64
ResolutionQuality             object
                              ...   
DistPermWater                float64
DistAgriLand                 float64
PercSoilClay                 float64
MinDayLength                 float64
VarDayLength                 float64
Length: 64, dtype: object

In [9]:
#Overview of statistics of the occurence dataset
occurence.describe()

,Occurrence_ID,AustraliaUTMGrid10Km_Lat,AustraliaUTMGrid10Km_Long,TotalNbOccurrencesPerUTMGrid,PtsIntensityEstimate_AllDataPerUTMGrid,PtsIntensityEstimate_OneDataPerUTMGrid,Year,Month,Day,Season,...,A_Prec_AvgAutumn30Yr,A_Prec_AvgSummer30Yr,A_Prec_AvgSpring30Yr,A_Prec_AvgWinter30Yr,VegeType,DistPermWater,DistAgriLand,PercSoilClay,MinDayLength,VarDayLength
count,689265.000000,689265.000000,689265.000000,689265.000000,689265.000000,689265.000000,672491.000000,663609.000000,619017.000000,662196.000000,...,649970.000000,649970.000000,649970.000000,649970.000000,688138.000000,688192.000000,688192.000000,688187.000000,688218.000000,688218.000000
mean,344634.160893,-34.024533,140.060762,44617.545375,0.862699,0.784935,2006.903548,4.855558,16.056586,2.235945,...,26.837895,27.838683,37.351001,38.334278,9.120894,3.897174,21.990139,25.285854,9.877882,2.477478
std,198975.628600,2.007720,3.253958,35380.316340,0.257677,0.097632,6.011869,2.953220,9.097762,0.906756,...,13.824578,14.454527,14.228456,17.741774,3.150844,5.360618,80.580640,6.108147,0.169346,0.368136
min,1.000000,-43.490000,113.050000,1.000000,0.000000,0.020000,1760.000000,0.000000,1.000000,1.000000,...,8.110000,5.310000,0.850000,1.230000,1.000000,0.000000,0.000000,5.000000,8.950000,0.250000
25%,172317.000000,-34.150000,139.050000,5779.000000,0.960000,0.770000,2007.000000,3.000000,9.000000,2.000000,...,18.710000,22.250000,27.680000,24.450000,10.000000,1.780000,1.570000,20.900000,9.870000,2.310000
50%,344633.000000,-34.150000,139.150000,45089.000000,0.980000,0.780000,2009.000000,4.000000,13.000000,2.000000,...,22.840000,23.520000,32.780000,35.970000,11.000000,3.360000,6.760000,28.100000,9.870000,2.480000
75%,516949.000000,-33.250000,139.450000,69866.000000,1.000000,0.800000,2009.000000,7.000000,24.000000,3.000000,...,29.750000,24.230000,42.730000,48.520000,11.000000,4.010000,20.240000,29.990000,9.950000,2.480000
max,689285.000000,-12.350000,153.650000,104045.000000,1.000000,1.000000,2015.000000,30.000000,31.000000,4.000000,...,447.450000,474.840000,251.570000,286.800000,13.000000,170.370000,915.020000,57.550000,11.400000,4.940000


In [10]:
#Overview of statistics of the abundance dataset
abundance.describe()

,SpotData_ID,AustraliaUTMGrid10Km_Long,AustraliaUTMGrid10Km_Lat,TotalNbTransectsPerUTMGrid,StudyRegions_ID,VisitNb,TotalNbReplicates,Year,Month,Day,...,A_Prec_AvgAutumn30Yr,A_Prec_AvgSummer30Yr,A_Prec_AvgSpring30Yr,A_Prec_AvgWinter30Yr,VegeType,DistPermWater,DistAgriLand,PercSoilClay,MinDayLength,VarDayLength
count,51241.000000,51241.000000,51241.000000,51241.000000,51241.000000,51241.000000,51241.000000,51214.000000,45754.000000,44565.000000,...,48687.000000,48687.000000,48687.000000,48687.000000,51241.000000,51241.000000,51241.000000,51241.000000,51241.000000,51241.000000
mean,25621.000000,139.446311,-35.493201,25.495911,14252.725337,16.832810,1.999219,2000.630628,6.640359,15.272635,...,42.637415,39.350071,52.176293,60.147965,8.856658,3.739985,32.049651,18.923074,9.736499,2.835243
std,14792.146909,10.155581,3.771567,20.404179,29585.293853,24.548531,0.705421,9.001740,3.457822,8.712367,...,19.932521,19.228277,25.149581,33.800933,3.512874,5.408652,113.835489,7.780317,0.329687,0.757181
min,1.000000,114.950000,-43.250000,1.000000,1.000000,1.000000,1.000000,1965.000000,1.000000,1.000000,...,10.500000,9.240000,10.040000,7.390000,1.000000,0.010000,0.070000,5.000000,8.970000,0.930000
25%,12811.000000,138.750000,-37.650000,7.000000,3745.000000,5.000000,2.000000,1998.000000,4.000000,8.000000,...,27.060000,24.660000,30.910000,40.440000,7.000000,0.380000,0.900000,14.720000,9.560000,2.360000
50%,25621.000000,143.250000,-36.150000,25.000000,4396.000000,10.000000,2.000000,2000.000000,6.000000,15.000000,...,42.070000,36.390000,53.420000,54.540000,11.000000,2.100000,2.240000,18.080000,9.690000,2.880000
75%,38431.000000,145.550000,-33.550000,35.000000,4436.000000,20.000000,2.000000,2008.000000,10.000000,22.000000,...,51.600000,49.960000,65.370000,70.650000,11.000000,5.210000,11.330000,24.420000,9.920000,3.220000
max,51241.000000,152.650000,-22.750000,84.000000,149692.000000,227.000000,9.000000,2015.000000,12.000000,31.000000,...,149.300000,158.550000,175.660000,227.980000,12.000000,52.940000,716.450000,50.000000,10.740000,4.870000


In [11]:
#Looking at the last 20 rows of the occurence dataset
occurence.tail(20)

,Occurrence_ID,AustraliaUTMGrid10Km_ID,AustraliaUTMGrid10Km_Lat,AustraliaUTMGrid10Km_Long,ResolutionQuality,TotalNbOccurrencesPerUTMGrid,PtsIntensityEstimate_AllDataPerUTMGrid,PtsIntensityEstimate_OneDataPerUTMGrid,AustraliaState,DataType,...,A_Prec_AvgAutumn30Yr,A_Prec_AvgSummer30Yr,A_Prec_AvgSpring30Yr,A_Prec_AvgWinter30Yr,VegeType,DistPermWater,DistAgriLand,PercSoilClay,MinDayLength,VarDayLength
689245,689265,55H1476,-34.45,147.55,More_than_15km,1,0.28,0.93,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689246,689266,55H4165,-38.93,146.44,More_than_15km,1,0.30,0.85,VIC,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689247,689267,54J4475,-31.45,141.45,More_than_15km,1,0.75,0.81,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689248,689268,53H1010,-33.65,136.95,More_than_15km,1,0.85,0.62,QLD,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689249,689269,55H0350,-32.55,148.95,More_than_15km,3,0.16,0.80,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689250,689270,50H1841,-35.05,118.05,More_than_15km,1,0.00,0.14,NT,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689251,689271,55H0417,-32.65,149.65,More_than_15km,1,0.13,0.76,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689252,689272,56H1090,-33.85,150.95,More_than_15km,20,0.09,0.69,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689253,689273,55H0898,-33.45,149.75,More_than_15km,21,0.12,0.76,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689254,689275,56H1569,-34.65,150.85,More_than_15km,9,0.10,0.68,WA,Expert_Observation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#Looking at the last 20 rows of the abundance dataset
abundance.tail(20)

,SpotData_ID,AustraliaUTMGrid10Km_ID,AustraliaUTMGrid10Km_Long,AustraliaUTMGrid10Km_Lat,ResolutionQuality,TotalNbTransectsPerUTMGrid,AustraliaState,StudyRegions_ID,DataSourceEntry_ID,DataSourceName,...,A_Prec_AvgAutumn30Yr,A_Prec_AvgSummer30Yr,A_Prec_AvgSpring30Yr,A_Prec_AvgWinter30Yr,VegeType,DistPermWater,DistAgriLand,PercSoilClay,MinDayLength,VarDayLength
51221,51222,54H0007,138.65,-32.05,Less_than_5km,1,SA,3719,Yednalue_18,SA_YednalueVenus,...,21.78,26.31,30.84,30.09,3,4.04,17.28,29.42,10.05,2.10
51222,51223,54H0007,138.65,-32.05,Less_than_5km,1,SA,3719,Yednalue_19,SA_YednalueVenus,...,21.78,26.31,30.84,30.09,3,4.04,17.28,29.42,10.05,2.10
51223,51224,54H0007,138.65,-32.05,Less_than_5km,1,SA,3719,Yednalue_20,SA_YednalueVenus,...,21.78,26.31,30.84,30.09,3,4.04,17.28,29.42,10.05,2.10
51224,51225,54H0007,138.65,-32.05,Less_than_5km,1,SA,3719,Yednalue_21,SA_YednalueVenus,...,21.78,26.31,30.84,30.09,3,4.04,17.28,29.42,10.05,2.10
51225,51226,54H0007,138.65,-32.05,Less_than_5km,1,SA,3719,Yednalue_22,SA_YednalueVenus,...,21.78,26.31,30.84,30.09,3,4.04,17.28,29.42,10.05,2.10
51226,51227,54H0007,138.65,-32.05,Less_than_5km,1,SA,3719,Yednalue_23,SA_YednalueVenus,...,21.78,26.31,30.84,30.09,3,4.04,17.28,29.42,10.05,2.10
51227,51228,54H0007,138.65,-32.05,Less_than_5km,1,SA,3719,Yednalue_24,SA_YednalueVenus,...,21.78,26.31,30.84,30.09,3,4.04,17.28,29.42,10.05,2.10
51228,51229,54J4747,138.65,-31.95,Between_10km_and_15km,1,SA,3720,Yednalue_25,SA_YednalueVenus,...,22.39,27.30,31.59,31.74,4,4.52,14.11,28.91,10.06,2.09
51229,51230,54J4747,138.65,-31.95,Between_10km_and_15km,1,SA,3720,Yednalue_26,SA_YednalueVenus,...,22.39,27.30,31.59,31.74,4,4.52,14.11,28.91,10.06,2.09
51230,51231,54J4747,138.65,-31.95,Between_10km_and_15km,1,SA,3720,Yednalue_27,SA_YednalueVenus,...,22.39,27.30,31.59,31.74,4,4.52,14.11,28.91,10.06,2.09


In [13]:
# Summary of the datasets

# Occurence dataset is sightings of rabbits by various means. This ranges
# between accounts from experts, active citizens, and people around
# warren locations. A warren is an area that rabbits congregate and 
# dig tunnels. 90% of rabbit sightings in this dataset are from people
# around warren locations (Roy-Dufresne 5).

# Abundance dataset is sightings of rabbits by transects. A transect is
# a trail a couple km's in length. Researchers go through the trail at 
# night, drive a car slowly while directing flashlights on the sides of 
# the transect. The number of rabbits spotted per km is recorded. There
# are different transect so each transect has an id.

In [17]:
# There ~10 weather. Is this from each particular day?
# Answer: From how I see it, it's saying that weather data is accounted
# for 30 years from 1976 to 2005. Therefore, this only has a value IF an
# occurence took place in between these years. If not, it has an empty value.

# Approximately 6% (39295/649970) of the weather data is NaN 
print(pd.isna(occurence[['A_Psea_Avg30Yr']]).value_counts())
print(pd.isna(occurence[['A_TAvg_Avg30Yr']]).value_counts())
print(pd.isna(occurence[['A_TMax_Avg30Yr']]).value_counts())
print(pd.isna(occurence[['A_TMin_Avg30Yr']]).value_counts())
print(pd.isna(occurence[['A_TSea_Avg30Yr']]).value_counts())
print(pd.isna(occurence[['A_TWet_Avg30Yr']]).value_counts())
print(pd.isna(occurence[['A_TWrm_Avg30Yr']]).value_counts())

A_Psea_Avg30Yr
False             649970
True               39295
dtype: int64
A_TAvg_Avg30Yr
False             649970
True               39295
dtype: int64
A_TMax_Avg30Yr
False             649970
True               39295
dtype: int64
A_TMin_Avg30Yr
False             649970
True               39295
dtype: int64
A_TSea_Avg30Yr
False             649970
True               39295
dtype: int64
A_TWet_Avg30Yr
False             649970
True               39295
dtype: int64
A_TWrm_Avg30Yr
False             649970
True               39295
dtype: int64


In [18]:
# Does each row say exactly how many rabbits were spotted?
# Answer: The paper mentions that in the Abundance dataset, yes. It is in
# the variable RabbitsPerKm. There appears to be no precise number feature
# for the occurence dataset. Its just that a person saw a rabbit(s).


# Any features that don't make sense?
# Answer: The QA variables are a little more difficult to understand.
# They assess the accuracy of each observation. A lot of detail given
# to these features in the paper.


# How do we determine population growth?
# Answer: Although the original goal was to determine population growth, 
# this is a strict metric. Since each occurence is a daily sighting of 
# rabbit(s), the same rabbit may be spotted on two separate days. However,
# since we are just starting out w/ our datasets, we'll assume each sighting
# is a unique rabbit. 
# At its most basic level, a population growth model is as such:
# The input variables are Year and Season. The target variable is 
# 'total occurences' for that year in that season. This variable 
# must be created and calculated in the abundance dataset.
# Later on in our project, we'll take into account weather variables,
# certain parts of Australia, and distance to geographic features.


# How will latitude longitude coordinates be utilized?
# Answer: k-Nearest neighbors. Create clusters each w/ a unique id.
# Recognize rabbit sighting hotspots for a particular day of the year.